# Day 12
## Part 1
Use [pyrsistent](https://pyrsistent.readthedocs.io/en/latest/)'s hashable vectors to check the paths.

In [20]:
from pyrsistent import pvector
from collections import defaultdict

def parse_data(s):
    data = defaultdict(list)
    for line in s.strip().splitlines():
        fields = line.split('-')
        data[fields[0].strip()].append(fields[1].strip())
        data[fields[1].strip()].append(fields[0].strip())
    return data
    
test_string = '''dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc
'''

test_data = parse_data(test_string)
test_data

defaultdict(list,
            {'dc': ['end', 'start', 'HN', 'LN', 'kj'],
             'end': ['dc', 'HN'],
             'HN': ['start', 'dc', 'end', 'kj'],
             'start': ['HN', 'kj', 'dc'],
             'kj': ['start', 'sa', 'HN', 'dc'],
             'LN': ['dc'],
             'sa': ['kj']})

In [22]:
def part_1(data):
    complete_paths = set()
    incomplete_paths = {pvector(['start', link]) for link in data['start']}
    while incomplete_paths:
        p = incomplete_paths.pop()
        for next_node in data[p[-1]]:
            if not (next_node.islower() and next_node in p):
                if next_node == 'end':
                    complete_paths.add(p.append('end'))
                else:
                    incomplete_paths.add(p.append(next_node))
    return len(complete_paths)

assert part_1(test_data) == 19

In [24]:
test_data_2 = parse_data('''
fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW
''')

assert part_1(test_data_2) == 226

In [25]:
data = parse_data(open('input', 'r').read())
part_1(data)

4749

## Part 2

The requirement that only one small cave can be visited at most twice makes things a bit fiddly. 

In [71]:
def valid_path(path):
    latest_node = path[-1]
    if path.count('start') > 1:
        return False
    elif latest_node.islower() and path.count(latest_node) > 2:
        return False
    else:
        previous_lcase_counts = {path[:-1].count(n) for n in set(path[:-1]) if n.islower()}
        return not(
            latest_node.islower() 
            and path.count(latest_node) > 1 
            and any(c > 1 for c in previous_lcase_counts)
        )

def part_2(data):
    complete_paths = set()
    incomplete_paths = {pvector(['start', link]) for link in data['start']}
    # visited_paths = set()
    while incomplete_paths:
        p = incomplete_paths.pop()
        for next_node in data[p[-1]]:
            incomplete_path = p.append(next_node)
            if next_node == 'end':
                complete_paths.add(incomplete_path)
            elif valid_path(incomplete_path): # and incomplete_path not in visited_paths:
                incomplete_paths.add(incomplete_path)
                #visited_paths.add(incomplete_path)
    return len(complete_paths)

assert part_2(test_data) == 103

In [72]:
part_2(data)


123054

That was tortuous and the code's really slow, for some reason.

In [69]:
%%timeit

part_2(data)

4.09 s ± 26.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
